In [2]:
# Date: Sept 9, 2024
# Author: Sahar El Abbadi
# Purpose: compare Song et al emissions factors 

In [3]:
# Setup 
import pandas as pd
import pathlib

pd.set_option('display.width', 1000)  # Set width to prevent line breaking


# Load Song et al spreadsheet 

song_ef = pd.read_excel(pathlib.PurePath('02_clean_data', 'song_EFs.xlsx'))
display(song_ef)

# Check entries in sahar_classification
print('Sahar classification:')
print(song_ef['sahar_classification'].unique())
ef_columns = ['BNR', 'Nitrification', 'Organics removal']


print('\nFacility level scale:')
print(song_ef['facility_scale'].unique())

print('\nMeasurement level scale:')
print(song_ef['measurement_scale'].unique())

,song_index,paper,year,water_line_description,facility_scale,process,sahar_notes,sahar_classification,measurement_scale,NH4_removal_rate,Reported_EF,Reported_EF_Unit,EF1_percentTNload,EF2_perm3
0,1,"Aboobakar et al., 2013",2013,plug-flow nitrifying lane preceded by a small ...,Full-scale,A/O,"Checked previously in De Haas et al. ""Conventi...",Nitrification,Bioreactor,0.993421,0.036,% gN2O-N/g TN load,0.036,0.016200
1,2,"Ahn et al., 2010",2010,A variant of the single reactor high-activity ...,Full-scale,PN,nitritation / SHARON - not representative of m...,Remove,Sidestream,NaN,0.240,% gN2O-N/g TKN load,0.240,2.789656
2,3,"Ahn et al., 2010",2010,A variant of the single reactor high-activity ...,Full-scale,PN,nitritation / SHARON - not representative of m...,Remove,Sidestream,NaN,0.540,% gN2O-N/g TKN load,0.540,7.578434
3,4,"Ahn et al., 2010",2010,plug flow (4 aeration passes),Full-scale,CAS,Used classification from De Haas et al previou...,Organics removal,Bioreactor,NaN,1.800,% gN2O-N/g TKN load,1.800,0.455058
4,5,"Ahn et al., 2010",2010,step-feed BNR (A/O-A/O-A/O-A/O),Full-scale,Step-feed,Used classification from De Haas et al previou...,BNR,Bioreactor,NaN,1.600,% gN2O-N/g TKN load,1.600,0.451825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,408,"Zhou et al., 2019",2019,OD,Pilot-scale,OD,TN in effluent is ~20 mg / L in all operationa...,Nitrification,Bioreactor,0.998200,0.028,% gN2O-N/g TN load,0.028,0.014000
408,409,"Zhou et al., 2019",2019,OD,Pilot-scale,OD,TN in effluent is ~20 mg / L in all operationa...,Nitrification,Bioreactor,0.998200,0.025,% gN2O-N/g TN load,0.025,0.012500
409,410,"Zhou et al., 2019",2019,OD,Pilot-scale,OD,TN in effluent is ~20 mg / L in all operationa...,Nitrification,Bioreactor,0.994400,0.102,% gN2O-N/g TN load,0.102,0.051000
410,411,"Zhou et al., 2019",2019,OD,Pilot-scale,OD,TN in effluent is ~20 mg / L in all operationa...,Nitrification,Bioreactor,0.982400,0.608,% gN2O-N/g TN load,0.608,0.304000


Sahar classification:
['Nitrification' 'Remove' 'Organics removal' 'BNR' 'Sludge' 'Lagoon']

Facility level scale:
['Full-scale' 'Pilot-scale']

Measurement level scale:
['Bioreactor' 'Sidestream' 'Plant scale' 'Post-nitrification' 'Sludge']


# Emissions factors estimates 

Calculate EFs to confirm I'm getting the same numbers in Python when compared with what I previously calculated.

1) All measurement data  

In [4]:
# EF for all facilities

ef_all = song_ef.groupby('sahar_classification')['EF1_percentTNload'].mean() # Calc EF with all values (no filtering based on scale)
ef_all.name = 'All facilities'
print('Emission factors for all facilities, classified by removal objective:\n')
print(ef_all[ef_columns])

Emission factors for all facilities, classified by removal objective:

sahar_classification
BNR                 1.371811
Nitrification       0.996246
Organics removal    0.277755
Name: All facilities, dtype: float64


2) Plant scale only (facility is "full-scale", so excluding pilot facilities) 

In [5]:
song_ef_full_scale = song_ef[song_ef['facility_scale']=='Full-scale'] # df with only full-scale facilities 

ef_full_scale = song_ef_full_scale.groupby('sahar_classification')['EF1_percentTNload'].mean()
ef_full_scale.name = 'Full-scale facilities only'

print('Emission factors for full-scale facilities only, classified by removal objective:\n')
print(ef_full_scale[ef_columns])

Emission factors for full-scale facilities only, classified by removal objective:

sahar_classification
BNR                 1.236654
Nitrification       1.116478
Organics removal    0.277755
Name: Full-scale facilities only, dtype: float64


3) Measurement scale is plant scale only, exclude bioreactor scale measurements  

In [6]:
song_ef_facility_scale = song_ef[song_ef['measurement_scale']=='Plant scale'] # df with only full-scale facilities 
ef_facility_scale = song_ef_facility_scale.groupby('sahar_classification')['EF1_percentTNload'].mean()
ef_facility_scale.name = 'Facility scale measurements'

print('Emission factors for measurements conducted at facility scale only, classified by removal objective:\n')
print(ef_facility_scale[ef_columns])

Emission factors for measurements conducted at facility scale only, classified by removal objective:

sahar_classification
BNR                 1.894652
Nitrification       0.590561
Organics removal    0.068152
Name: Facility scale measurements, dtype: float64


4) Measurement scale is plant or bioreactor scale

Excluded categories are "Sidestream", "Post-nitrification", and "Sludge" 


In [7]:
song_ef_facility_bioreactor_scale = song_ef[song_ef['measurement_scale'].isin(['Plant scale', 'Bioreactor'])] # df with only full-scale facilities 
ef_facility_bioreactor_scale = song_ef_facility_bioreactor_scale.groupby('sahar_classification')['EF1_percentTNload'].mean()
ef_facility_bioreactor_scale.name = 'Plant or bioreactor scale measurement'

print('Emission factors for measurements conducted at facility or bioreactor level scale, classified by removal objective:\n')
print(ef_facility_bioreactor_scale[ef_columns])

Emission factors for measurements conducted at facility or bioreactor level scale, classified by removal objective:

sahar_classification
BNR                 1.293735
Nitrification       1.114170
Organics removal    0.277755
Name: Plant or bioreactor scale measurement, dtype: float64


5) Measurement scale is plant OR bioreactor, and facility scale is full scale

In [9]:
# Same df as above (measurement scale is plant or bioreactor), but only include full-scale facilities (no pilot scale) 

song_ef_facility_bioreactor_scale_full = song_ef_facility_bioreactor_scale[song_ef_facility_bioreactor_scale['facility_scale'].isin(['Full-scale'])] # df with only full-scale facilities 
# ef_facility_bioreactor_scale_full = song_ef_facility_bioreactor_scale_full.groupby('sahar_classification')['EF1_percentTNload'].mean()

# Group by 'sahar_classification' and calculate both the mean and count
ef_facility_bioreactor_scale_full = song_ef_facility_bioreactor_scale_full.groupby('sahar_classification')['EF1_percentTNload'].agg(
    mean_value='mean', 
    sample_size='size'
)

ef_facility_bioreactor_scale_full.columns = ['Mean EF', 'Sample Size']

ef_facility_bioreactor_scale_full.name = 'Plant or bioreactor at full scale'
print('Emission factors for measurements conducted at facility or bioreactor level scale at full-scale facilities, classified by removal objective:\n')
display(ef_facility_bioreactor_scale_full)

print(f'Total number of measurements: {len(song_ef_facility_bioreactor_scale_full)}')
print(f'Fraction of measurements from BNR facilities: {221/len(song_ef_facility_bioreactor_scale_full)}')

Emission factors for measurements conducted at facility or bioreactor level scale at full-scale facilities, classified by removal objective:


,Mean EF,Sample Size
sahar_classification,,
BNR,1.123996,221
Lagoon,-0.004946,2
Nitrification,1.321354,33
Organics removal,0.277755,22
Remove,0.750287,3


Total number of measurements: 281
Fraction of measurements from BNR facilities: 0.7864768683274022


## Combine all EFs 

Relevant dataframes from above: 
ef_all
ef_full_scale
ef_facility_scale
ef_facility_bioreactor_scale
ef_facility_bioreactor_scale_full

In [ ]:
# Combine the all results into one DataFrame
ef_combined = pd.concat([ef_all[ef_columns], ef_full_scale[ef_columns], ef_facility_scale[ef_columns], ef_facility_bioreactor_scale[ef_columns], ef_facility_bioreactor_scale_full[ef_columns]], axis=1)

# Print the combined DataFrame for comparison
print('Emission factors comparison:')
print(ef_combined)

ef_save_path = pathlib.PurePath('04_results', 'EF_combined.csv')
ef_combined.to_csv(ef_save_path, index=True)

# Song et al facility classification 

Song et al classify measurements based on their processes as either Non-BNR processes or BNR processes (Fig 3b from the manuscript): 

Non-BNR processes: TF, Lagoon, CAS
BNR processes: IA, EA, BAF, OD, A/O, MLE, A2O, AGS, S+A, Bardenpho, UCT, Step-feed, SBR, MBR


In [ ]:
# Check the process column to ensure that all entries align with those included in Fig 3
print(song_ef['process'].unique())


In [ ]:
# Classify TF, CAS as non-BNR process and all else classify as BNR process 

# Define the function to classify based on 'process'
def classify_process_song(process):
    if process in ['TF', 'CAS']:
        return 'Organics removal'
    elif process == 'Lagoon':
        return 'Lagoon'
    elif process == 'Sludge':
        return 'Sludge'
    else:
        return 'BNR'

song_ef['song_classification'] = song_ef['process'].apply(classify_process_song)

print(song_ef[['process', 'sahar_classification', 'song_classification']])


### Any mismatch between Song and El Abbadi classification 

Find any mismatch between my classification and that in Song et al 


In [ ]:
# Filter rows where 'song_classification' is not equal to 'sahar_classification'
mismatch_df = song_ef[song_ef['song_classification'] != song_ef['sahar_classification']]

# Display the mismatched rows
print(mismatch_df[['paper','song_classification', 'sahar_classification', 'EF1_percentTNload', 'sahar_notes']])

# Count the number of occurrences of each mismatch pair
mismatch_counts = mismatch_df.groupby(['sahar_classification', 'song_classification']).size().reset_index(name='count')

# Display the mismatch counts
print(mismatch_counts)
print(f"Total mismatches in classification: {mismatch_counts['count'].sum()}")

### Mismatch in organics removal assignment
Check to see which facilities are classified by Song et al as Organics removal and not classified as Organics removal in sahar_classification 

In [ ]:
# Filter the DataFrame to select facilities classified as organics removal by Song et al, and classified as something else by me 
mismatch_organics_classification = song_ef[(song_ef['song_classification'] == 'Organics removal') & (song_ef['sahar_classification'] != 'Organics removal')]

print(mismatch_organics_classification[['paper','song_classification', 'sahar_classification', 'EF1_percentTNload', 'sahar_notes']])

# Check effect of changing emissions factor

Very quick and dirty test to see how changes to EFs might impact overall results 

Inputs: new EF, affected flow

In [ ]:
def mgd_to_mm3y(mgd): 
    gal_per_m3 = 264.172
    days_per_year = 365
    return mgd /gal_per_m3 * days_per_year

def calc_new_notal_n2o(ef_nit=0, flow_nit=0, ef_bnr=0, flow_bnr=0, ef_org=0, flow_org=0):
    
    # flow input is in MGD 
    # EF input is in % (g N2O-N / g N influent) 
    
    # Assumed N concentration in wastewater 
    total_n = 40 # units mg / L or kg / M3 
    
    # GWP potential of nitrous oxide 
    gwp_n2o = 273 
    
    # Current emissions factors in latest round of results
    ef_current_nit = 2.7
    ef_current_bnr = 0.82
    ef_current_org = 0.36 
    
    
    # Difference between current and proposed emissions factors 
    ef_nit_dif = ef_nit - ef_current_nit
    ef_bnr_dif = ef_bnr - ef_current_bnr
    ef_org_dif = ef_org - ef_current_org
    
    # Convert flow from MGD to MM3 / year 
    flow_nit_mm3 = mgd_to_mm3y(flow_nit)
    flow_bnr_mm3 = mgd_to_mm3y(flow_bnr)
    flow_org_mm3 = mgd_to_mm3y(flow_org)
    
    change_n2o_n = (ef_nit_dif * flow_nit_mm3 + ef_bnr * flow_bnr_mm3 + ef_org_dif * flow_org_mm3)*total_n # units: kg N / year 
    
    # kg N2O / kg N
    kg_n2O_per_kg_N = 44 / 28
    
    change_co2eq = change_n2o_n * kg_n2O_per_kg_N * gwp_n2o # units: kg CO2 eq / year 
    
    # Convert to MMT / year
    kg_per_ton = 1000
    change_co2eq = change_co2eq / kg_per_ton / 1E6
    
    return change_co2eq




    
    